##### Quantum Data Science 2022/2023
### Lecture 1 - Introduction to quantum machine learning - The swap test and the swap test classifier 

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

<!-- no toc -->
### Contents 

1. [Qiskit recap](#recap)
2. [Important functions and libraries](#important)
3. [EXERCISE 1 - basis states probabilities](#ex1)
4. [EXERCISE 2 - SWAP Test](#swaptest)
5. [EXERCISE 3 - Binary classifier using the SWAP Test](#swaptestclassifier)

#### 1. Qiskit basic operations <a id="recap"></a>

```python

#create quantum register with n qubits
qr = QuantumRegister(n, name="qr")

#assign register to quantum circuit 
qc = QuantumCircuit(qr)

#create classical register to store output of measurements
cr = ClassicalRegister(n, name="cr")

#add new quantum/classical register to already created quantum circuit
new_qr = QuantumRegister(n, name="new_qr")
qc.add_register(new_qr)

#SINGLE-QUBIT GATES

#not gate 
qc.x(qr[0]) #qc.operation(qr) applies to all qubits
#phase gate
qc.z(qr[0])
#complex phase gate
qc.y(qr[0])
#hadamard gate
qc.h(qr[0])
#S gate - pi/2 phase shift
qc.s(qr[0])
#T gate - pi/4 phase shift
qc.t(qr[0])
#y rotation gate
qc.ry(angle,qr[0])

<p align="center">
  <img width="650" height="350" src="images/blochsphere.png">
</p>

```python 

#arbitrary rotation gate 
qc.u(theta,phi,lmbda, qr[0])


#TWO/THREE QUBIT GATES

#controlled not gate 
qc.cx(qr[0],qr[1])
#controlled phase gate
qc.cz(qr[0],qr[1])
#toffoli gate 
qc.ccx(qr[0],qr[1],qr[2])
#multi control toffoli 
qc.mct(controls, target)
#swap gate
qc.swap(qr[0],qr[1])
#control swap gate
qc.cswap(qr[0],qr[1],qr[2])

In [ ]:
#Visualize the circuit
qr = QuantumRegister(5)
qc = QuantumCircuit(qr)
qc.mct(qr[:4],qr[4])
qc.measure_all()
qc.draw(output="mpl")

```python

#MEASUREMENT

qc.measure(qc, cr) #measures all qubits one-to-one to classical register

#if one wants to measure every qubit cr is not needed
qc.measure_all()

#BARRIER - cleaner circuit 

qc.barrier()

#PREPARE INITIAL STATE FROM STATE VECTOR

qc.initialize(state_vector, qr) #here https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.initialize.html




### Note on qiskit ordering of qubits 

Qiskit uses little-endian , left qubit is the most significant . Important for easy binary to decimal conversion. 

<p align="center">
  <img width="650" height="400" src="images/littleendian.png">
</p>


#### 2. Libraries and functions <a id="important"></a>

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

##### Function for executing a quantum circuit 

In [ ]:
def execute_circuit(qc, shots=1024, device=None):
    if device is None:
        device = Aer.get_backend('qasm_simulator')
        #state vector simulator could also be used
        #device = Aer.get_backend('state_vector_simulator')
    else:
        device = device

    counts = device.run(qc, shots=shots).result().get_counts()
    
    return counts

### Quantum Computing is probabilistic in nature
Outcomes of measurements are "usually" not deterministic 

<p align="center">
  <img width="600" height="400" src="images/quantum_prob.png">
</p>



#### 3. <span style="color: red;">EXERCISE 1</span> - Basis states probabilities <a id="ex1"></a>
Implement a function *basis_states_probs* that executes an arbitrary quantum circuit and returns the probability of each of the basis states.

Test the function by executing the quantum circuit for the GHZ state with 5 qubits $ |\psi\rangle = \frac{1}{\sqrt{2}} (|00000\rangle + |11111\rangle)$

In [ ]:
### SOLUTION ###
 
def basis_states_probs(counts, shots=1024, n_qubits=1):
   probs = []
   basis_states = [np.binary_repr(i,width=n_qubits) for i in range(2**n_qubits)]

   ### YOUR CODE HERE ###
   for state in basis_states:
         if state in counts:
            probs.append(counts[state]/shots)
         else:
            probs.append(0)
   return probs

In [ ]:
# prepare the GHZ STATE #
n_qubits = 8
qc = QuantumCircuit(n_qubits)
qc.h(0)
for i in range(n_qubits-1):
    qc.cx(i,i+1)
qc.measure_all()

counts = execute_circuit(qc,shots=1024)
probs = basis_states_probs(counts, shots=1024, n_qubits=n_qubits)

# histogram of the results
plot_histogram(counts)

# for n in range(2**n_qubits):
#     print("probability |{}> - {}".format(np.binary_repr(n,width=n_qubits),probs[n]))

#### 4 - <span style="color: red;">EXERCISE 2</span> - SWAP Test <a id="swaptest"></a>

<p align="center">
  <img width="500" height="250" src="images/swaptestt.png">
</p>

Using the swap test, prove analytically that for arbitrary states $|\psi\rangle$ and $|\phi\rangle$, the probability of measuring the ancilla in state $|0\rangle$ is given by: 
$$ P(0) = \frac{1}{2} + \frac{1}{2} |\langle \psi | \phi \rangle |^2$$ 

What should be the overlap between the states $|\psi\rangle = |+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ and state $|\psi\rangle = |-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$ . Compute the overlap between the states.

In [ ]:
### SOLUTION ###

def swap_test(qc: QuantumCircuit, ancilla, psi, phi, cr, n_qubits=1):

    overlap = 0 
    
    ### YOUR CODE ###
    qc.h(ancilla)
    qc.cswap(ancilla, psi, phi)
    qc.h(ancilla)
    qc.measure(ancilla, 0)

    counts = execute_circuit(qc, shots=1024)
    overlap = counts['0']/1024
    overlap = (overlap - 1/2) * 2

    return overlap, qc

#prepare states |+> and |-> and compute overlap using swap_test function
qc = QuantumRegister(3)
cr = ClassicalRegister(1)
qc = QuantumCircuit(qc, cr)
psi = 1
phi = 2
ancilla = 0
qc.x(psi)
qc.h(psi)
qc.h(phi)

overlap, qc= swap_test(qc, ancilla, psi, phi, cr)
print("Overlap |<a|b>|^2 - {}".format(overlap))
qc.draw(output="mpl")

### <span style="color: red;">EXERCISE 2</span> - A binary classifier using the swap test

Consider the problem of **binary classification** - You have a labelled dataset $D = \{ (x_1,y_1) , \dots , (x_N,y_N)\}$ where $x$ corresponds to a point and $y$ to the respective label. As the name suggests, in binary classification $y$ takes only two values. For instance, suppose that points $x$ are images of cats vs dogs. In this case, $y$ corresponds to the label "cat" or "dog". Mathematically speaking we could use "cat" $\mapsto 0$ and "dog" $\mapsto 1$.

Consider now a dataset of single qubit quantum states $D = \{ (x_1,y_1) , \dots , (x_N,y_N)\}$ where $x$ corresponds to a quantum state and $y$ to the label. $0$ in the case of the state being closer to state $|0\rangle$ and $1$ if the state is closer to state $|1\rangle$.

<p align="center">
  <img width="800" height="450" src="images/swaptestclassifier.png"> 
</p>


The classification for each new unlabelled quantum state in $D'$ can be done by measuring the distance to every quantum state in $D$. The distance is calculated individually for each label and averaged. The label that has the smallest average distance should be the assigned label to the new quantum state in $D'$. Let $\overline{d_{i}^{0}}$ be the average distance between the new quantum state $|\phi_i\rangle$ and every quantum state of label $0$ in $D$, $|\psi_j^0\rangle$ and $\overline{d_{i}^{1}}$ be the average distance between the new quantum state $i$ and every quantum state of label $1$, $|\psi_j^1\rangle$. The distance can be calculated using the swap test presented above, where the distance is given by the overlap between two quantum states.

$$ \overline{d_{i}^{0}} = \frac{1}{N} \sum_{j=1}^{N} |\langle \psi_j^0 | \phi_i \rangle|^2$$

$$ \overline{d_{i}^{1}} = \frac{1}{N} \sum_{j=1}^{N} |\langle \psi_j^1 | \phi_i \rangle|^2$$

The label for the new quantum state is assigned dependently on the average distances. 

$$ y_i = \left\{
\begin{array}{ll}
      0 & \overline{d_{i}^{0}} \geq \overline{d_{i}^{1}} \\
      1 &  otherwise \\
\end{array} 
\right. $$

Let's start by loading the labelled dataset, $d$ and unlabelled dataset $d\_$ with quantum states that we want to classify.

Load datasets from file q_dataset.npy
    
- d is the dataset with quantum state vector and labels [((a_1, b_1), label1) , ...]
- d_bloch is the dataset with bloch vector representations to plot in the bloch sphere [((x_1, y_1, z_1), label1) , ...] 
- d_ is the unlabelled dataset with only quantum state vectors [((a_1, b_1)]
- d__bloch is the unlabelled dataset with bloch vector representations to plot in the bloch sphere [(x_1, y_1, z_1) , ...]

In [ ]:
with open("q_dataset.npy", 'rb') as f:
    d = np.load(f, allow_pickle=True)
    d_bloch = np.load(f, allow_pickle=True)    
    d_ = np.load(f, allow_pickle=True)
    d__bloch = np.load(f, allow_pickle=True)

In [36]:
d[0]

array([list([(-0.10844879521576177+0j), (0.7489253938837458+0.6537198277644071j)]),
       1], dtype=object)

Plot the data i.e., plot the single-qubit quantum states in the bloch sphere to understand your data. Use **d_bloch**

In [ ]:
from kaleidoscope import bloch_sphere

vec = []
colors = []

for i in d_bloch:
    if i[1]:
        colors.append("#AA00FF")
    else:
        colors.append("#AA00F")

    vec.append(i[0])

for i in d__bloch:
    colors.append("#00FF00")
    vec.append(i)

bloch_sphere(points=vec, points_color=colors)


##### Use the swap test to classify the quantum states present in $d\_$

You can choose to either use the euclidean distance or the original inner product distance. 

**Note:** Use the qiskit function **initialize** to prepare arbitrary quantum states from statevector

- x = QuantumRegister(1)
- x_ = QuantumRegister(1)
- qc = QuntumCircuit(x,x_)

- qc.initialize(d[0], x)
- qc.initialize(d_[0], x_)

##### 4.1 - create a function that outputs the distance between two quantum states: 

In [37]:
def swap_distance(qc: QuantumCircuit, ancilla: int, x_i: int, x_j: int, cr: int) -> float:
    overlap = 0 
    ### YOUR CODE ###
    qc.h(ancilla)
    qc.cswap(ancilla, x_i, x_j)
    qc.h(ancilla)
    qc.measure(ancilla, cr)

    counts = execute_circuit(qc, shots=1024)
    overlap = counts['0']/1024
    overlap = (overlap - 1/2) * 2

    return overlap

In [38]:
### SOLUTION ###

def distance(qc, ancilla, x, x_, cr) -> float:

    distance = 0

    ### YOUR CODE HERE ###
    qc.h(ancilla)
    qc.cswap(ancilla, x, x_)
    qc.h(ancilla)
    qc.measure(ancilla, 0)

    counts = execute_circuit(qc, shots=1024)
    distance = counts['0']/1024
    distance = (distance - 1/2) * 2
    
    return distance 

##### 4.2 - create a function that outputs the average distance for elements of a given class

In [39]:
def swap_avg_distance(set_points: np.ndarray, not_set_point: np.ndarray) -> np.ndarray:
    avg_distance = np.zeros(2)

    for set_point in set_points:
        an = QuantumRegister(1)
        x = QuantumRegister(1)
        x_ = QuantumRegister(1)
        cr = ClassicalRegister(1)
        qc = QuantumCircuit(an, x, x_, cr)
        qc.initialize(set_point[0], x)
        qc.initialize(not_set_point, x_)
        ancilla = 0
        xi = 1
        xj = 2
        cr = 0
        avg_distance[set_point[1]] += swap_distance(qc, ancilla, xi, xj, cr)

    return avg_distance / len(set_points)

In [40]:
### SOLUTION ### 

def avg_distance(d, x_):

    avg_distance = np.zeros(2)

    qr = QuantumRegister(3)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr, cr)
    for x in d:
        qc.initialize(x,qr[1])
        qc.initialize(x_,qr[2])
        ancilla = 0
        avg_distance[0] += distance(qc, ancilla, 1, 2, cr)
        avg_distance[1] += distance(qc, ancilla, 2, 1, cr)
        
    return avg_distance

#### 4.3 - Create function that classifies each quantum state in $d\_$

In [103]:
from typing import List
def swap_classify(set_points: np.ndarray, not_set_point: np.ndarray) -> List[int]:
    lables = []

    for not_set_point in not_set_point:
        avg_distance = swap_avg_distance(set_points, not_set_point)
        if avg_distance[0] < avg_distance[1]:
            lables.append(0)
        else:
            lables.append(1)

    return lables

In [104]:
### SOLUTION ###

def classify(d,d_):

    labels = []

    for x_i in d_:

        avg_dist = avg_distance(d, x_i)

        ### YOUR CODE HERE ###
        if avg_dist[0] < avg_dist[1]:
            labels.append(0)
        else:
            labels.append(1)

    return labels

#### Visualize the obtained classification by plotting again in the bloch sphere

In [145]:
vec = []
colors = []

labels = swap_classify(d,d_)
new_d__bloch = []

for i in range(len(d__bloch)):
    new_d__bloch.append([d__bloch[i],labels[i]])

for i in new_d__bloch:
    if i[1]:
        #purple
        colors.append("#AA00FF")
    else:
        #red
        colors.append("#AA00F")

    vec.append(i[0])

bloch_sphere(points=vec, points_color=colors)


### Homework
- To classify each point we need to compare it with every single point in the dataset. 
- What if instead of computing distances between every point, we had a centroid for the label, for instance the average distance between each datapoint in the dataset? Then, to classify new points, we would only need to compare the distance to centroid instead of every point. Experiment and compare the results with the previous classifier.

In [144]:
def swap_classify(set_points: np.ndarray, not_set_points: np.ndarray) -> np.ndarray:
    avg_1 = np.array(list(map(np.array, set_points[set_points[:,1] == 1][:,0]))).mean(axis=0)
    avg_2 = np.array(list(map(np.array, set_points[set_points[:,1] == 0][:,0]))).mean(axis=0)

    avg_1 /= np.linalg.norm(avg_1)
    avg_2 /= np.linalg.norm(avg_2)

    labels = []
    for not_set_point in not_set_points:
        an = QuantumRegister(1)
        x = QuantumRegister(1)
        x_ = QuantumRegister(1)
        cr = ClassicalRegister(1)
        qc = QuantumCircuit(an, x, x_, cr)
        qc.initialize(avg_1, x)
        qc.initialize(not_set_point, x_)
        ancilla = 0
        xi = 1
        xj = 2
        cr = 0
        dist_1 = swap_distance(qc, 0, 1, 2, 0)
        an = QuantumRegister(1)
        x = QuantumRegister(1)
        x_ = QuantumRegister(1)
        cr = ClassicalRegister(1)
        qc = QuantumCircuit(an, x, x_, cr)
        qc.initialize(avg_2, x)
        qc.initialize(not_set_point, x_)
        ancilla = 0
        xi = 1
        xj = 2
        cr = 0
        dist_2 = swap_distance(qc, 0, 1, 2, 0)
        if dist_1 < dist_2:
            labels.append(1)
        else:
            labels.append(0)

    return labels


In [135]:
# avg_1 = np.mean(np.array(list(map(np.array, d[d[:,1] == 1][:,0])))) 
# avg_2 = np.mean(np.array(list(map(np.array, d[d[:,1] == 0][:,0]))))

# avg_1, avg_2
a = np.array(list(map(np.array, d[d[:,1] == 1][:,0]))).mean(axis=0)
a /= np.linalg.norm(a)

array([-0.10125869+0.j        , -0.07769137+0.25076591j])